# Spopularity

---


# Requesting new token

First request is for updating the token, if needed. Credentials are stored in `credentials.json` -file.


In [ ]:
import json
import spotipy
import time
import requests


def get_token():
    client_id = ""
    client_secret = ""
    token = ""
    expiration = 0

    with open("credentials.json", "r") as f:
        credentials = json.load(f)
        client_id = credentials["client_id"]
        client_secret = credentials["client_secret"]
        token = credentials["token"]
        expiration = credentials["expiration"]

    if not token or expiration < time.time():
        token = spotipy.oauth2.SpotifyClientCredentials(
            client_id=client_id, client_secret=client_secret
        )
        token = token.get_access_token(as_dict=False)
        credentials["token"] = token
        credentials["expiration"] = int(time.time() + 3600)

        with open("credentials.json", "w") as f:
            f.write(json.dumps(credentials))

    if token:
        print("Token acquired.")
    return token


s = spotipy.Spotify(get_token())

# Requests to RapidAPI Billboard top 100 weekly chart

**Not needed for requesting track data from Spotify**

Requires registering an account. **Limit for monthly requests is 30 (use wisely).**

Headers (credentials) also in `credentials.json`.


In [ ]:
url = "https://billboard-api2.p.rapidapi.com/hot-100"
querystring = {"date": "2023-05-11", "range": "1-100"}

file = open("credentials.json", "r")
credentials = json.load(file)
rapid_api_key, rapid_api_host = (
    credentials["rapid-api-key"],
    credentials["rapid-api-host"],
)
file.close()

headers = {"X-RapidAPI-Key": rapid_api_key, "X-RapidAPI-Host": rapid_api_host}

# response = requests.get(url, headers=headers, params=querystring)

# print(response.json())

In [ ]:
import pandas as pd

genres = [
    "new-release",
    "pop",
    "pop-film",
    "power-pop",
    "synth-pop",
]


def read_top_100(
    filename: str = "20231028-top100.csv", skip_header_line: bool = True
) -> tuple[list[str], list[str]]:
    """Read the file of latest Billboard top 100 list artists and titles."""
    chart_artists = []
    chart_tracks = []
    with open(filename, "r") as chart_file:
        data = chart_file.read()
        i = 0
        lines = data.split("\n")
        for line in lines:
            i += 1
            if (skip_header_line and i == 1) or i == len(lines):
                continue
            l = line.split(";")
            if l[0] and (l[0][1:-1] not in chart_artists):
                chart_artists.append(l[0][1:-1])
            if l[1] and (l[1][1:-1] not in chart_tracks):
                chart_tracks.append(l[1][1:-1])
    return (chart_artists, chart_tracks)


def search(search_terms: list, search_type: str = "artist", limit: int = 3) -> list:
    ids = []
    for item in search_terms:
        search_result = s.search(item, limit=limit, type=search_type)
        type_plural = search_type + "s"
        for res in search_result[type_plural]["items"]:
            if res["id"] not in ids:
                ids.append(res["id"])

    filename = search_type + "_ids.csv"
    with open(filename, "w") as file:
        for id in ids:
            file.write(id)
            file.write("\n")

    return ids


def get_recommendations(
    seed_artist: list = [],
    seed_genres: list = [],
    seed_tracks: list = [],
    limit: int = 10,
) -> None:
    # Spotipy recommendations does not seem to work
    # recommendations = s.recommendations(
    #     seed_artists=seed_artists,
    #     seed_genres=genres,
    #     seed_tracks=seed_tracks,
    #     limit=limit,
    # )
    url = f"https://api.spotify.com/v1/recommendations?limit={limit}"
    if seed_genres:
        g = "&seed_genres=" + "%2C".join(seed_genres)
        url += g
    headers = {"Authorization": f"Bearer {get_token()}"}
    res = requests.get(url=url, headers=headers)
    # print(res.json()["tracks"][0]["id"])
    result = res.json()
    recommendations = read_file("recommendations.csv")
    print(recommendations)
    for i in range(limit):
        new_rec = result["tracks"][i]["id"]
        if new_rec not in recommendations:
            print("new rec not in file")
            recommendations.append(new_rec)
    write_file("recommendations.csv", recommendations, "w")


def write_file(filename: str, data: list, write_mode: str = "w") -> None:
    with open(filename, str(write_mode)) as file:
        for i in range(len(data)):
            file.write(data[i])
            if i < len(data) - 1:
                file.write("\n")


def read_file(filename: str) -> list:
    ids = []
    try:
        with open(filename, "r") as file:
            data = file.read()
            for line in data.split("\n"):
                ids.append(line)
    except FileNotFoundError:
        return []
    return ids


# (chart_artists, chart_tracks) = read_top_100()
# search(chart_artists)
# search(chart_tracks, search_type="track")

seed_artists = read_file("artist_ids.csv")
seed_tracks = read_file("track_ids.csv")
# get_recommendations(seed_artists, genres, seed_tracks)
get_recommendations(seed_artist=None, seed_genres=genres, seed_tracks=None, limit=5)


billboard_top_5 = [
    "1kuGVB7EU95pJObxwvfwKS",
    "2IGMVunIBsBLtEQyoI1Mu7",
    "4KULAymBBJcPRpk1yO4dOG",
    "1Lo0QY9cvc8sUB2vnIOxDT",
    "1BxfuPKGuaTgP7aM0Bbdwr",
]

track_details = s.tracks(billboard_top_5)

tracks = []
for t in track_details["tracks"]:
    audio_features = s.audio_features(t["id"])
    track_name = t["name"]
    track_id = t["id"]
    artist_name = t["artists"][0]["name"]
    popularity = t["popularity"]

    audio_features = s.audio_features(t["id"])[0]
    danceability = audio_features["danceability"]
    energy = audio_features["energy"]
    key = audio_features["key"]
    loudness = audio_features["loudness"]
    mode = audio_features["mode"]
    speechiness = audio_features["speechiness"]
    acousticness = audio_features["acousticness"]
    instrumentalness = audio_features["instrumentalness"]
    liveness = audio_features["liveness"]
    valence = audio_features["valence"]
    tempo = audio_features["tempo"]
    type = audio_features["type"]
    id = audio_features["id"]
    uri = audio_features["uri"]
    track_href = audio_features["track_href"]
    analysis_url = audio_features["analysis_url"]
    duration_ms = audio_features["duration_ms"]
    time_signature = audio_features["time_signature"]

    # audio_analysis = s.audio_analysis(t["id"])

    tracks.append(
        [
            track_name,
            track_id,
            artist_name,
            popularity,
            danceability,
            energy,
            key,
            loudness,
            mode,
            speechiness,
            acousticness,
            instrumentalness,
            liveness,
            valence,
            tempo,
            type,
            id,
            uri,
            track_href,
            analysis_url,
            duration_ms,
            time_signature,
        ]
    )

df = pd.DataFrame(tracks)

# print(df.head())
# print(df.describe())